<a href="https://colab.research.google.com/github/jishnu1711/crowd-detection/blob/main/crowd_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this is for all the dependencies and libraries

In [2]:
# Clone YOLOv5 and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 17410, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 17410 (delta 63), reused 22 (delta 22), pack-reused 17320 (from 3)
Receiving objects: 100% (17410/17410), 16.31 MiB | 21.97 MiB/s, done.
Resolving deltas: 100% (11929/11929), done.
/content/yolov5/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.0 MB/s eta 0:00:00
Setup complete. Using torch 2.6.0+cu124 (CPU)


this is to import annotated data set from robo flow

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="2CF5NR1QQDNvxFwkmSNJ")
project = rf.workspace("spoorthimodel").project("crowd-detection-xhpiu")
version = project.version(3)
dataset = version.download("yolov5")

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

this is to test the train the model

In [ ]:
!python train.py --img 416 --batch 16 --epochs 150 --data Crowd-detection-3/data.yaml --weights yolov5s.pt --cache


this is to import all the test photo frames for testing

In [ ]:
import os
from google.colab import files
import shutil

# Path to your test folder
test_folder = '/content/yolov5/Crowd-detection-3/test'

# Create the folder if it doesn't exist
os.makedirs(test_folder, exist_ok=True)

# Upload files from local device
uploaded = files.upload()

# Move uploaded files to the test folder
for file_name in uploaded.keys():
    shutil.move(file_name, os.path.join(test_folder, file_name))
    print(f"✅ Moved {file_name} to {test_folder}")


In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 416 --source /content/yolov5/Crowd-detection-3/test --save-txt


In [ ]:
# Path to YOLOv5 label files after detecting on images
labels_path = '/content/yolov5/runs/detect/exp/labels'

density_class = lambda count: (
    'Low' if count <= 10 else 'Medium' if count <= 25 else 'High'
)

image_densities = {}

for file in sorted(os.listdir(labels_path)):
    if file.endswith('.txt'):
        with open(os.path.join(labels_path, file), 'r') as f:
            count = 0
            for line in f:
                parts = line.strip().split()
                if len(parts) == 6:
                    conf = float(parts[5])
                    if conf >= 0.35:
                        count += 1
            image_densities[file] = {
                "count": count,
                "density": density_class(count)
            }

# Print results
print("🖼️ Crowd Density per Frame (Image Input):\n")
for image, data in image_densities.items():
    print(f"{image}: {data['count']} person(s) → {data['density']} Density")


In [ ]:
uploaded = files.upload()  # upload your .mp4 video
!python detect.py \
  --weights best.pt \
  --img 416 \
  --source your_video.mp4 \
  --save-txt \
  --save-conf


In [ ]:
# Path to YOLOv5 detection label outputs from video
labels_path = '/content/yolov5/runs/detect/exp/labels'

density_class = lambda count: (
    'Low' if count <= 10 else 'Medium' if count <= 25 else 'High'
)

frame_densities = {}

for file in sorted(os.listdir(labels_path)):
    if file.endswith('.txt'):
        with open(os.path.join(labels_path, file), 'r') as f:
            count = 0
            for line in f:
                parts = line.strip().split()
                if len(parts) == 6:  # includes confidence
                    conf = float(parts[5])
                    if conf >= 0.35:
                        count += 1
            frame_densities[file] = {
                "count": count,
                "density": density_class(count)
            }

# Print results
print("🎥 Crowd Density per Frame (Video Input):\n")
for frame, data in frame_densities.items():
    print(f"{frame}: {data['count']} person(s) → {data['density']} Density")

